In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Function to parse each line of the file
def parse_line(line):
    # Split the line by commas for the titles or by spaces for the data
    parts = line.strip().split(',') if ',' in line else line.strip().split()
    return parts

# Specify the names of the files to search for
target_files = ['flow_cut_to_failure.vtk', 'transport_cut_to_failure.vtk']
n = 135

# Arrays to store the read data
h2aq = np.zeros([n+1, 1])
h2saq = np.zeros([n+1, 1])
h2g = np.zeros([n+1, 1])

In [24]:
for i in range(1,n+1):
    # Get the current directory
    current_dir = './../Goe_Base/run_' + str(i) + '/'

    # Search for the target files in the current directory
    for file in os.listdir(current_dir):
        if file in target_files:
            print("Failure found in folder:", current_dir)
            break  # Exit the loop if either file is found
        else:
            read_folder = True
            
    if not os.path.exists('./../Goe_Base/run_' + str(i) + '/pflotran_h2-012.vtk'):
            print(f"File does not exist: {'./../Goe_Base/run_' + str(i) + '/pflotran_h2-012.vtk'}")
            continue
    else:
        file_path_h2 = './../Goe_Base/run_' + str(i) + '/pflotran_h2-012.vtk'
        #============
        # Read H2(aq)
        # ============
        # Initialize an empty list to store the numbers
        numbers_list_h2 = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2 = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS Total_H2(aq)' in line:
                    start_reading_h2 = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2 = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2_all = np.array(numbers_list_h2)
        numbers_array_h2 = numbers_array_h2_all[numbers_array_h2_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2 = cell_vol_gr*numbers_array_sat*numbers_array_h2
        total_h2 = np.sum(cell_h2)

        # print("Total H2(aq) is " + str(total_h2) + " moles")
        h2aq[i, 0] = total_h2

        # ============
        # Read H2S(aq)
         # ============
        file_path_h2s = './../Goe_Base/run_' + str(i) + '/pflotran_h2-012.vtk'
        # Initialize an empty list to store the numbers
        numbers_list_h2s = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2s = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2s, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS H2S(aq)' in line:
                    start_reading_h2s = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2s:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2s = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2s.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2s_all = np.array(numbers_list_h2s)
        numbers_array_h2s = numbers_array_h2s_all[(numbers_array_h2s_all <= 1.0) & (numbers_array_h2s_all >= 2e-8)]
        # numbers_array_sat_all = np.array(numbers_list_sat)
        # numbers_array_sat = numbers_array_sat_all[numbers_array_h2s_all <= 1.0 and numbers_array_h2s_all >= 2e-8]

        # cell_vol_gr = 16*16*20*0.2*1000

        # # Display the first few numbers to verify
        # cell_h2s = cell_vol_gr*numbers_array_sat*numbers_array_h2s
        # total_h2s = np.sum(cell_h2s)

        # print("Total H2S(aq) is: " + str(total_h2s) + " moles")
        h2saq[i, 0] = np.average(numbers_array_h2s) # total_h2s

        # ==========
        # Read H2(g)
        # ==========
        # Define the path to your .dat file
        file_path_h2g = './../Goe_Base/run_' + str(i) + '/pflotran_h2-mas.dat'

        # Read the file and parse each line
        with open(file_path_h2g, 'r') as file:
            lines = file.readlines()

        # Parse the column titles
        column_titles = parse_line(lines[0])

        # Initialize a list to hold the parsed data
        data = []

        # Parse each line of data
        for line in lines[1:]:  # Skip the first line with column titles
            parsed_line = parse_line(line)
            if parsed_line:  # Ensure the line is not empty
                data.append(parsed_line)

        # Create a DataFrame from the parsed data
        # The length of the longest row is used to ensure all rows have the same length
        max_length = max(len(row) for row in data)
        df = pd.DataFrame(data, columns=column_titles[:max_length])

        df.columns = df.columns.str.strip('"')
        last_number_global_h2 = df["Global H2(g) [mol]"].iloc[11]
        last_number_global_h2 = pd.to_numeric(last_number_global_h2, errors='coerce')
        h2g[i, 0] = last_number_global_h2
        print("The H2(g) [mol] after 1Y is: ", last_number_global_h2)


The H2(g) [mol] after 1Y is:  90566012700.0
The H2(g) [mol] after 1Y is:  89969333700.0
The H2(g) [mol] after 1Y is:  89045484600.0
The H2(g) [mol] after 1Y is:  88301240200.0
The H2(g) [mol] after 1Y is:  87729547600.0
The H2(g) [mol] after 1Y is:  87574379300.0
The H2(g) [mol] after 1Y is:  87459700100.0
The H2(g) [mol] after 1Y is:  87420229100.0
The H2(g) [mol] after 1Y is:  87400178000.0
The H2(g) [mol] after 1Y is:  87388004800.0
The H2(g) [mol] after 1Y is:  87379807100.0
The H2(g) [mol] after 1Y is:  87373898100.0
The H2(g) [mol] after 1Y is:  87369428700.0
The H2(g) [mol] after 1Y is:  87365924800.0
The H2(g) [mol] after 1Y is:  87363100300.0
The H2(g) [mol] after 1Y is:  90565865600.0
The H2(g) [mol] after 1Y is:  89004242500.0
The H2(g) [mol] after 1Y is:  87576883200.0
The H2(g) [mol] after 1Y is:  87476887200.0
The H2(g) [mol] after 1Y is:  87432771400.0
The H2(g) [mol] after 1Y is:  87399080500.0
The H2(g) [mol] after 1Y is:  87358643600.0
The H2(g) [mol] after 1Y is:  87

In [25]:
print(h2aq)
print(h2saq)
print(h2g)

np.savetxt('./H2AQ_Base_1Y.txt', h2aq, fmt='%d', delimiter=', ')
np.savetxt('./H2SAQ_Base_1Y.txt', h2saq, fmt='%.10e', delimiter=', ')
np.savetxt('./H2G_Base_1Y.txt', h2g, fmt='%d', delimiter=', ')

[[0.00000000e+00]
 [7.57613537e+08]
 [7.52504011e+08]
 [7.44553802e+08]
 [7.38135261e+08]
 [7.33226466e+08]
 [7.31960204e+08]
 [7.30974810e+08]
 [7.30635677e+08]
 [7.30463417e+08]
 [7.30358853e+08]
 [7.30288445e+08]
 [7.30237703e+08]
 [7.30199327e+08]
 [7.30169246e+08]
 [7.30145002e+08]
 [7.57615577e+08]
 [7.44205358e+08]
 [7.31989980e+08]
 [7.31130811e+08]
 [7.30751760e+08]
 [7.30462277e+08]
 [7.30114844e+08]
 [7.29993106e+08]
 [7.29928955e+08]
 [7.29888342e+08]
 [7.29859793e+08]
 [7.29838313e+08]
 [7.29821391e+08]
 [7.29807605e+08]
 [7.29796087e+08]
 [7.57617695e+08]
 [7.37331518e+08]
 [7.31100441e+08]
 [7.30565131e+08]
 [7.30340098e+08]
 [7.30162475e+08]
 [7.29944347e+08]
 [7.29862937e+08]
 [7.29816962e+08]
 [7.29785863e+08]
 [7.29762658e+08]
 [7.29744266e+08]
 [7.29729111e+08]
 [7.29716272e+08]
 [7.29705172e+08]
 [7.53043295e+08]
 [7.50141009e+08]
 [7.47356199e+08]
 [7.46236660e+08]
 [7.45661261e+08]
 [7.45188211e+08]
 [7.44589491e+08]
 [7.44378687e+08]
 [7.44271249e+08]
 [7.442062